In [ ]:
%run conceptors.py

In [ ]:
%pylab inline

In [ ]:

# making some waveform patterns that we might use
pSaw = lambda n: (round(n % waveLengthSamples) / waveLengthSamples * 2) - 1.0
pPulse = lambda n: (((n % waveLengthSamples) < (waveLengthSamples * 0.5)) * 2) - 1.0
pSine2 = lambda n: (np.sin(n) * np.sin((n+np.pi/4)/6))
pSine3 = lambda n: (np.sin(n) * np.sin((n/4)/6)/6)
pJ1 = lambda n: 1 * np.sin(2 * np.pi * n / 3.1504531)
pJ1b = lambda n: 1 * np.sin(n/2) ** 1

period2 = 2
rawp = np.random.randn(period2)
# rawp = np.array([1.1929,2.6856]);
maxVal = np.max(rawp)
minVal = np.min(rawp)
rp = 0.5 * (2 * (rawp - minVal) / (maxVal - minVal) - 1);
pJ2 = lambda n: rp[np.mod(n, period2 )]
pTri = lambda n,p: (((n % p) >= (p/2)) * ((p/2) - (n % (p/2))) + ((n % p) < (p/2)) * (n % (p/2))) * (2/p)



In [ ]:
figsize(20,4)
# plot([pJ1b(x) for x in arange(15)])
# plot([pJ1(x) for x in arange(15)])
# plot([pJ2(x) for x in arange(15)])
plot([pSine2(x) for x in arange(15)])

# plot([pTri(x,10) for x in arange(10)])

In [ ]:
params = {'N':30, 
          'NetSR':0.6, # was 1.6; scaling for internal weights: according to ESN tutorial, this one should actually be < 1
          'NetinpScaling':1.6, # scaling for input weights
          'BiasScaling':0.3, # scaling for bias
          'TychonovAlpha':0.0001,
          'TychonovAlphaReadout':0.0001,
          'washoutLength':100, 'learnLength':500, 
          'learnLengthWout':500, 
          'recallTestLength':100,
          'alphas':np.array([12.0,24.0,36.0]),
          'patts':np.array([pJ1b, pJ2,pSine2])
}

net1 = makeNetwork(params)


In [ ]:
net2 = makeNetwork(params)
net3 = makeNetwork(params)


In [ ]:
morphValues = [1,0,0,1,0,0,1,0,0]

def onMorph(ind,f):
    global morphValues
#     print("onMorph", f)
    morphValues[ind] = f
    return morphValues

def onExit():
    print( "exiting" )
    global keepRunning
    keepRunning = False
    return True

oscserver = makeOSCServer(onMorph,onExit)

In [ ]:
import time

keepRunning = True

#run a network
x1 = 0.5 * np.random.randn(net1['p']['N'],1)
x2 = 0.5 * np.random.randn(net2['p']['N'],1)
x3 = 0.5 * np.random.randn(net3['p']['N'],1)

while keepRunning:
#     print( morphValues )
    # net 1
    output1, x1 = conceptor_mix_step( net1, x1, morphValues[0:3] )
    oscserver.send_value( "/output/1", output1[0][0] )
    oscserver.send_array( "/x/1", x1.flatten() )
    
    # net 2
    output2, x2 = conceptor_mix_step( net2, x2, morphValues[3:6] )
    oscserver.send_value( "/output/2", output2[0][0] )
    oscserver.send_array( "/x/2", x2.flatten() )

    # net 3
    output3, x3 = conceptor_mix_step( net3, x3, morphValues[6:9] )
    oscserver.send_value( "/output/3", output3[0][0] )
    oscserver.send_array( "/x/3", x3.flatten() )
    
    time.sleep(0.01)
    

In [ ]:
def conceptor_mix_step( net, x, morphvalues, oversample=1 ):
    ind = 0
    C = np.zeros( (net['p']['N'] , net['p']['N'] ) )
    for i_morph in morphvalues:
        C = C + (net['Cs'][0,ind].dot( i_morph ))
        ind = ind + 1
    Wsr = net['W'].dot(1.2)
    for i_oversample in range( oversample ):
        x = np.tanh(Wsr.dot(x) + net['Wbias'])
        x = C.dot(x)
    output = net1['Wout'].dot(x)
    return output, x


In [ ]:
print( net1['p']['N'] )
print( morphValues[0:3], morphValues[3:6], morphValues[6:9])
morphFactors = morphValues[0:3]
a, b = conceptor_mix_step( net1, x1, morphFactors )
print( a )
print( b )

In [ ]:
morphValues

In [ ]:
#test OSC
try:
    target = Address(57400)
except AddressError as err:
    print (str(err))
    sys.exit()

# send message "/foo/message1" with int, float and string arguments
send(target, "/morph", 0.3)

In [ ]:
a = [ 0, 2, 4]
oscserver.send_array("/morph", a)